In [1]:
import sys, os, json, copy, sklearn, shap, gc, joblib, copy, time
import pandas as pd
import numpy as np
import lightgbm as lgb
from lightgbm import LGBMClassifier
import pickle as pkl
from tqdm import tqdm
from smart_open import open
import seaborn as sns
from sklearn.metrics import roc_curve,roc_auc_score, precision_score, recall_score, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from rdsutils.woe import WOE_Transform                                             
from rdsutils.metrics import get_pred_reports

pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', None)

sys.path.insert(1, "./")

%load_ext autoreload
%autoreload 2

print(lgb.__version__)

3.2.1


In [7]:
##

with open('s3://sofi-data-science/jxu/pl-gen4/prescreen/prescreen_final/woe_dict.pkl', "rb") as f:
    woe_dict_full = pkl.load(f)
    
imputer_path = 's3://sofi-data-science/jxu/pl-gen4/prescreen/prescreen_final/prescreen_impute_values.json'
with open(os.path.join(imputer_path), "r") as f:
    impute_vals = json.load(f)

In [11]:
f = 't11_tstu2752'
display(woe_dict_full[f])
impute_vals[f]

,%accts,min,max,woe,distance
0,0.04%,0.0,0.0,1.4296,1.2835
1,0.14%,1.0,24.0,0.7256,0.5795
2,0.05%,25.0,30.0,0.4571,0.3110
3,0.49%,31.0,89.0,0.4402,0.2941
4,0.24%,90.0,109.0,0.2997,0.1536
5,2.13%,110.0,297.0,0.1876,0.0415
6,0.51%,298.0,316.0,0.0417,0.1044
7,0.51%,317.0,350.0,-0.0299,0.1760
8,1.49%,351.0,461.0,-0.0434,0.1895
9,0.36%,462.0,487.0,-0.1402,0.2863


203.5